In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### 데이터 가져오기

In [2]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print(df_wine.shape)
df_wine.head()

(178, 14)


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


#### 학습을 위한 데이터 처리
* 와인 클래스 2와 클래스 3만 사용 (1은 제거)

In [3]:
# 클레스 라벨이 1인 데이터 제거
df_wine = df_wine[df_wine["Class label"]!=1]

#### 전처리

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = df_wine.drop(["Class label"], axis=1)
y = df_wine["Class label"]
print(X.shape, y.shape)

# 라벨 인코딩 {1, 2} -> {0, 1}
le = LabelEncoder()
y = le.fit_transform(y)

# 훈련 테스트데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(119, 13) (119,)


((95, 13), (24, 13), (95,), (24,))

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10,  #  트리의 개수
                            criterion="gini", 
                            max_depth=None, # 최대 분할 개수
                            min_samples_split=20,  # 노드를 분할하기 위한 최소한의 샘플 데이터 수, 과적합을 제어하는데 사용한다. Default = 2 → 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가한다.
                            min_samples_leaf=2,  #  분할하기위한 최소한의 샘플 데이터수  , 불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정 필요
                            min_weight_fraction_leaf=0, # min_sample_leaf와 같지만 가중치가 부여된 샘플 수에서의 비율
                            max_features="auto", # 최적의 분할을 위해 고려할 최대 피처 개수
                                                  #int형으로 지정 →피처 갯수 / float형으로 지정 →비중
                            max_leaf_nodes=None,  # 리프노드의 최대 개수
                            min_impurity_decrease=0,  #최소 불순도
                            bootstrap=True, # 복원추출 여부
                            oob_score=False, # out_of_bag 데이터를 test 데이터로 사용할것인가.
                            n_jobs=None,  # 적합성과 예측성을 위해 병렬로 실행할 작업 수
                            random_state=1, 
                            verbose=0,  # 실행 과정 출력 여부
                            warm_start=False, # 이전 호출의 솔루션을 재사용하여 합계에 더 많은 견적가를 추가
                            class_weight=None, # 클래스 가중치
                            ccp_alpha=0, 
                            max_samples=None # Train set의 크기
                            )

In [17]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(base_estimator=lr, # 분류모델
                            n_estimators=10, # 
                            max_samples=1.0, # 
                            max_features=1.0, # 
                                              #int형으로 지정 → 갯수 / float형으로 지정 → 비율
                            bootstrap=True, #
                            bootstrap_features=True, # 각 분류모델에 들어가는 샘플에 대해서 feature 중복 사용여부
                            oob_score=False, # 일반화 정확도를 줄이기 위해 밖의 샘플 사용 여부
                            warm_start=False, # 이전 호출의 솔루션을 재사용하여 합계에 더 많은 견적가를 추가
                            n_jobs=None, # 적합성과 예측성을 위해 병렬로 실행할 작업 수
                            random_state=1, # seed 고정
                            verbose=0  # 실행 과정 출력 여부
                            )

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver="liblinear")

#### k-겹 교차검증

In [18]:
from sklearn.model_selection import cross_val_score

models = ["rf", "lr"]

print('10-겹 교차검증:')
for model in models:
    scores = cross_val_score(estimator=eval(model),
                             X= X_train, y=y_train, cv=10, scoring="accuracy")
    print(f'{model} Acc : {scores.mean():.2f} (+/- {scores.std():.2f})')

10-겹 교차검증:
rf Acc : 0.97 (+/- 0.07)
lr Acc : 0.96 (+/- 0.05)


#### 스코어 계산

In [21]:
for model in models:
    eval(model).fit(X_train, y_train)
    train_score = eval(model).score(X_train, y_train)
    test_score = eval(model).score(X_train, y_train)
    
    print(f'{model} train Acc : {train_score:.2f}, test Acc :{test_score:.2f})')

rf train Acc : 1.00, test Acc :1.00)
lr train Acc : 0.99, test Acc :0.99)
